In [ ]:
import csv
import chars2vec
import re
import numpy as np
import keras.backend as K
from tensorflow.keras import datasets, layers, models, losses, callbacks
from sklearn.model_selection import train_test_split
import statistics

In [ ]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1 if len(predictions) == 0 else 0
    if len(predictions) == 0:
        return 0
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [ ]:
def read_data_text(filename):
    """Reads csv file with python, text."""
    data = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            data.append(row['text'])
    csvfile.close()
    return data

In [ ]:
def read_data_spans(filename):
    """Reads csv file with python, text."""
    data = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            if row['spans'] == '[]' or row['spans'] == []:
                data.append([])
            else:
                data.append([int(j) for j in row['spans'][1:-1].split(", ")])
    csvfile.close()
    return data

In [ ]:
texts = read_data_text('gdrive/My Drive/Colab Notebooks/Data/tsd_test_readable_text.csv')
spans = read_data_spans('gdrive/My Drive/Colab Notebooks/Data/tsd_test.csv')
spans